<a href="https://colab.research.google.com/github/ccc-frankfurt/Practical_ML_SS21/blob/main/week08/Autoencoders_MLPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoders in PyTorch
Here, we extend our previous MLP and CNN PyTorch Fashion-MNIST example to the unsupervised learning scenario with autoencoders. 

Before starting the notebook you should make sure that your runtime uses GPU acceleration. You can find the corresponding option under *runtime* and then *change runtime type*.

In [ ]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline

### Dataset class in PyTorch
Our dataset loader essentially remains the same as before. We do not remove the labels even though we do not need them for training of autoencoders. We will however use a small subset of the labels for semi-supervised training towards the end of the notebook. 

In [ ]:
import os
import struct
import gzip
import errno
import torch.utils.data
import torchvision.datasets as datasets


class FashionMNIST:
    """
    Fashion MNIST dataset featuring gray-scale 28x28 images of
    fashion items belonging to ten different classes.
    Dataloader adapted from MNIST.
    We do not define __getitem__ and __len__ in this class
    as we are using torch.utils.data.TensorDataSet which
    already implements these methods.

    Parameters:
        args (dict): Dictionary of (command line) arguments.
            Needs to contain batch_size (int) and workers(int).
        is_gpu (bool): True if CUDA is enabled.
            Sets value of pin_memory in DataLoader.

    Attributes:
        trainset (torch.utils.data.TensorDataset): Training set wrapper.
        valset (torch.utils.data.TensorDataset): Validation set wrapper.
        train_loader (torch.utils.data.DataLoader): Training set loader with shuffling.
        val_loader (torch.utils.data.DataLoader): Validation set loader.
    """

    def __init__(self, is_gpu, batch_size, workers):
        self.path = os.path.expanduser('datasets/FashionMNIST')
        self.__download()

        self.trainset, self.valset = self.get_dataset()

        self.train_loader, self.val_loader = self.get_dataset_loader(batch_size, workers, is_gpu)

        self.val_loader.dataset.class_to_idx = {'T-shirt/top': 0,
                                                'Trouser': 1,
                                                'Pullover': 2,
                                                'Dress': 3,
                                                'Coat': 4,
                                                'Sandal': 5,
                                                'Shirt': 6,
                                                'Sneaker': 7,
                                                'Bag': 8,
                                                'Ankle boot': 9}

    def __check_exists(self):
        """
        Checks if dataset has already been downloaded

        Returns:
             bool: True if downloaded dataset has been found
        """

        return os.path.exists(os.path.join(self.path, 'train-images-idx3-ubyte.gz')) and \
               os.path.exists(os.path.join(self.path, 'train-labels-idx1-ubyte.gz')) and \
               os.path.exists(os.path.join(self.path, 't10k-images-idx3-ubyte.gz')) and \
               os.path.exists(os.path.join(self.path, 't10k-labels-idx1-ubyte.gz'))

    def __download(self):
        """
        Downloads the Fashion-MNIST dataset from the web if dataset
        hasn't already been downloaded.
        """

        from six.moves import urllib

        if self.__check_exists():
            return

        print("Downloading FashionMNIST dataset")
        urls = [
            'https://cdn.rawgit.com/zalandoresearch/fashion-mnist/ed8e4f3b/data/fashion/train-images-idx3-ubyte.gz',
            'https://cdn.rawgit.com/zalandoresearch/fashion-mnist/ed8e4f3b/data/fashion/train-labels-idx1-ubyte.gz',
            'https://cdn.rawgit.com/zalandoresearch/fashion-mnist/ed8e4f3b/data/fashion/t10k-images-idx3-ubyte.gz',
            'https://cdn.rawgit.com/zalandoresearch/fashion-mnist/ed8e4f3b/data/fashion/t10k-labels-idx1-ubyte.gz',
        ]

        # download files
        try:
            os.makedirs(self.path)
        except OSError as e:
            if e.errno == errno.EEXIST:
                pass
            else:
                raise

        for url in urls:
            print('Downloading ' + url)
            data = urllib.request.urlopen(url)
            filename = url.rpartition('/')[2]
            file_path = os.path.join(self.path, filename)
            with open(file_path, 'wb') as f:
                f.write(data.read())

        print('Done!')

    def __get_fashion_mnist(self, path, kind='train'):
        """
        Load Fashion-MNIST data

        Parameters:
            path (str): Base directory path containing .gz files for
                the Fashion-MNIST dataset
            kind (str): Accepted types are 'train' and 't10k' for
                training and validation set stored in .gz files

        Returns:
            numpy.array: images, labels
        """

        labels_path = os.path.join(path,
                                   '%s-labels-idx1-ubyte.gz'
                                   % kind)
        images_path = os.path.join(path,
                                   '%s-images-idx3-ubyte.gz'
                                   % kind)

        with gzip.open(labels_path, 'rb') as lbpath:
            struct.unpack('>II', lbpath.read(8))
            labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

        with gzip.open(images_path, 'rb') as imgpath:
            struct.unpack(">IIII", imgpath.read(16))
            images = np.frombuffer(imgpath.read(), dtype=np.uint8).reshape(len(labels), 784)

        return images, labels

    def get_dataset(self):
        """
        Loads and wraps training and validation datasets

        Returns:
             torch.utils.data.TensorDataset: trainset, valset
        """

        x_train, y_train = self.__get_fashion_mnist(self.path, kind='train')
        x_val, y_val = self.__get_fashion_mnist(self.path, kind='t10k')

        # This is new with respect to our previous data loader
        # convert to torch tensors in range [0, 1]
        x_train = torch.from_numpy(x_train).float() / 255
        y_train = torch.from_numpy(y_train).long()
        x_val = torch.from_numpy(x_val).float() / 255
        y_val = torch.from_numpy(y_val).long()

        # resize flattened array of images for input to a CNN
        # we use the in-place variant of the resize function here
        x_train.resize_(x_train.size(0), 1, 28, 28)
        x_val.resize_(x_val.size(0), 1, 28, 28)

        # TensorDataset wrapper
        trainset = torch.utils.data.TensorDataset(x_train, y_train)
        valset = torch.utils.data.TensorDataset(x_val, y_val)

        return trainset, valset

    def get_dataset_loader(self, batch_size, workers, is_gpu):
        """
        Defines the dataset loader for wrapped dataset

        Parameters:
            batch_size (int): Defines the batch size in data loader
            workers (int): Number of parallel threads to be used by data loader
            is_gpu (bool): True if CUDA is enabled so pin_memory is set to True

        Returns:
             torch.utils.data.TensorDataset: trainset, valset
        """

        # multi-threaded data loaders
        train_loader = torch.utils.data.DataLoader(self.trainset, batch_size=batch_size, shuffle=True,
                                                   num_workers=workers, pin_memory=is_gpu, sampler=None)
        test_loader = torch.utils.data.DataLoader(self.valset, batch_size=batch_size, shuffle=True,
                                                  num_workers=workers, pin_memory=is_gpu, sampler=None)

        return train_loader, test_loader


Let's load the data and set the device to use. 

In [ ]:
# set a boolean flag that indicates whether a cuda capable GPU is available 
# we will need this for transferring our tensors to the device and 
# for persistent memory in the data loader
is_gpu = torch.cuda.is_available()
print("GPU is available:", is_gpu)
print("If you are receiving False, try setting your runtime to GPU")

# set the device to cuda if a GPU is available
device = torch.device("cuda" if is_gpu else "cpu")

# in contrast to our MLP from scratch notebook, we need to set the batch size already now
# this is because our data loader now requires it.
batch_size = 128
# we also set the amount of workers, i.e. parallel threads to use in our data loader
workers = 4

# We can now instantiate our dataset class 
dataset = FashionMNIST(is_gpu, batch_size, workers)

GPU is available: True
If you are receiving False, try setting your runtime to GPU


### The MLP based Autoencoder model in PyTorch
We extend our 2 hidden layer MLP to an autoencoder by adding a decoder. For convenience we let the decoder architecture (number and amount of units) remain the same as the encoder. Note that however, this is not necessary and could technically be different. 

Suitable hidden-layer sizes for this task could be 100 and 100, like in our last notebook. 
Because we are using an optimized GPU implementation, you are welcome and should try larger sizes to see the impact of neural network size (capacity) on our task!

From previous weeks we know that there is three ways to build our model: 

    1. Defining an nn.Sequential() container for encoder and decoder and calling them in the forward function.
    2. Defining all the layers and writing an encode and decode function that specify the execution of the individual layers. The encode and decode functions get called in the forward pass.
    3. Defining the layers and writing all the calls directly into the forward function. While this is a valid model, we strongly recommend against this, as for more involved models this gets more complicated to read and understand.  
    
Here we will use option 1 and we will see how to implement version 2 for the variational autoencoder next week.

In [ ]:
class AE_MLP(nn.Module):
    def __init__(self, img_size, latent_dim):
        super(AE_MLP, self).__init__()
        
        self.img_size = img_size
        
        # encoder
        self.encoder = nn.Sequential()
        
        # decoder
        self.decoder = nn.Sequential()

    def forward(self, x):
        # You will now have to use a view both to flatten the input
        x = x.view(...)
        z = self.encoder(...)
        x = self.decoder(...)
        # because the MLP has flattened, we now need to view the output back as an image
        x = x.view(...)
        return x

### Defining optimization criterion and optimizer
Similarly to our supervised sample we can now use the negative log-likelihood as a loss function and optimize it with a binary cross entropy to measure the reconstruction loss between input and output. 

As with the CrossEntropy function before PyTorch already implements a Sigmioid function in the BCEWithLogitsLoss and combines it directly in the loss! Alternatively we could just use the BCE loss function and add the sigmoid to the end of our decoder by hand or stick with a mean squared error MSELoss() function.

This time we will use an adaptive optimizer that we have learned about in the first lecture, called Adam: https://arxiv.org/abs/1412.6980. A typical learning rate for Adam will be 1e-4 or 1e-3 as the step size is adaptive.

In [ ]:
# Define optimizer and loss function (criterion)
img_size = 28
# the latent dimension defines how much the input will be compressed, 
# this is a hyper-parameter.
latent_dim = 20

# create an instance of the MLP based autoencoder and transfer the model to the device.
# Note that we do not necessarily need any custom weight initialization as PyTorch
# already uses the initialization schemes that we have previously learned about internally. 
model = AE_MLP(img_size, latent_dim).to(device)
# we can also print the model architecture
print(model)

# set the loss function
criterion = nn.BCEWithLogitsLoss().to(device)

# we can use advanced stochastic gradient descent algorithms 
# with regularization (weight-decay) or momentum
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

### Monitoring losses 

We will reuse our function to monitor loss averages. As we are now conducting unsupervised learning, we no longer have any function calculating accuracies.

In [ ]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### Training function
The training function needs to loop through the entire dataset in steps of mini-batches (for SGD). For each mini-batch the output of the model and losses are calculated and a *backward* pass is done to calculate gradients and an *optimizer step* is done in order to do the respective update to the model's weights. 

When the entire dataset has been processed once, one epoch of the training has been conducted. It is common to shuffle the dataset after each epoch. In contrast to our previous notebook from scratch, in this implementation this is handled by the "sampler" of the dataset loader. 

In [ ]:
def train(train_loader, model, criterion, optimizer, device):
    """
    Trains/updates the model for one epoch on the training dataset.

    Parameters:
        train_loader (torch.utils.data.DataLoader): The trainset dataloader
        model (torch.nn.module): Model to be trained
        criterion (torch.nn.criterion): Loss function
        optimizer (torch.optim.optimizer): optimizer instance like SGD or Adam
        device (string): cuda or cpu
    """

    # create an instance of the average meter to track losses
    losses = AverageMeter()

    # switch to train mode
    model.train()

    # iterate through the dataset loader
    # we can now discard the labels returned by our old data loader as we no longer need them
    for i, (..., ...) in enumerate(train_loader):
        # transfer inputs and targets to the GPU (if it is available)
        inp = ...
        target = ... 
        
        # you can make your autoencoder a denoising autoencoder by 
        # adding noise to the input, but not to the target!
        # To test the denoising autoencoder, uncomment the line below, 
        # where we add a small Gaussian noise to the original images
        #inp = inp + torch.randn(inp.size()).to(device) * 0.1
        
        # compute output, i.e. the model forward
        output = model(...)
        
        # calculate the loss
        loss = criterion(..., ...)

        # record loss
        losses.update(loss.item(), inp.size(0))

        # compute gradient and do the SGD step
        # we reset the optimizer with zero_grad to "flush" former gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print the loss every 100 mini-batches
        if i % 100 == 0:
            print('Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(loss=losses))

### Validation function
Validation is similar to the training loop, but on a separate dataset with the exception that no update to the weights is performed. This way we can monitor the generalization ability of our model and check whether it is overfitting (memorizing) the training dataset.  

In [ ]:
def validate(val_loader, model, criterion, device):
    """
    Evaluates/validates the model

    Parameters:
        val_loader (torch.utils.data.DataLoader): The validation or testset dataloader
        model (torch.nn.module): Model to be evaluated/validated
        criterion (torch.nn.criterion): Loss function
        device (string): cuda or cpu
    """

    # create an instance of the average meter to track losses
    losses = AverageMeter()

    # switch to evaluate mode 
    # (this would be important for e.g. dropout where stochasticity shouldn't be applied during testing)
    model.eval()

    # avoid computation of gradients and necessary storing of intermediate layer activations
    with torch.no_grad():
        # iterate through the dataset loader
        # we can now discard the labels returned by our old data loader as we no longer need them
        for i, (..., ...) in enumerate(val_loader):
            # transfer to device
            inp = ...
            target = ...

            # compute output
            output = model(...)

            # compute loss
            loss = criterion(..., ...)

            # record loss
            losses.update(loss.item(), inp.size(0))
            
            # visualize only one full mini-batch
            if i == (len(val_loader) - 2):
                # let us also visualize the last mini-batch of images and reconstructions
                # we can use the torchvision utility to make grids of images
                print("Original images")
                imgs = torchvision.utils.make_grid(inp.cpu(), nrow=int(math.sqrt(inp.size(0))), padding=5)
                npimgs = imgs.numpy()
                # when using matplotlib the color channels are expected to be in the third 
                # instead of the first dimension -> tranpose
                plt.imshow(np.transpose(npimgs, (1,2,0)))
                plt.show()
                
                # do the same for reconstructions
                print("Reconstructed images")
                recons = torchvision.utils.make_grid(torch.sigmoid(output).cpu(), nrow=int(math.sqrt(inp.size(0))), padding=5)
                npimgs = recons.numpy()
                plt.imshow(np.transpose(npimgs, (1,2,0)))
                plt.show()


    print('Loss {loss.val:.4f} ({loss.avg:.4f})'.format(loss=losses))

### Running the training of the model
Let's optimize this model for 20 epochs and check at every epoch how we are doing on our validation set. As defined above, we also visualize some images and their reconstructions.

Depending on your model definition and optimizer you might experience over-fitting!

In [ ]:
total_epochs = 20
for epoch in range(total_epochs):
    print("EPOCH:", epoch + 1)
    print("TRAIN")
    train(...)
    print("VALIDATION")
    validate(...)

## From unsupervised learning to semi-supervised learning
Now that we have seen that our model was able to successfuly learn how to compress and decompress the data in a completely unsupervised fashion, we can try to make use of the learned representations

### Making use of learned representations
Assume that we have a dataset where a majority of images is not labelled, so we can not directly use supervised end-to-end learning as encountered in the last weeks. However, our autoencoder's encoder learns a feature description that best described the seen dataset. 

We can use the feature representation of our autoencoder's encoder and now simply train a classifier (e.g. a simple linear combination) on top with the few labels that we have. This way the classifier can learn how the autoencoder's extracted generic feature representations map to specific classes.

For this purpose let us do the following four things:

    1. Let us again define an accuracy function to measure our classification progress
    2. Let us define supervised training and validation functions, where we now first compute the already trained autoencder's decoder and then compute a classifier
    3. Let us define a single layer linear classifier 
    4. Let us define a new optimizer instance for only the classifier, i.e. the weights of the autoencoder remain fixed.
    5. Let us take half of the validation data (i.e. only 5% of the entire datasets) to train the supervised scenario with few labels and test on the remaining 50% of the validation set.

### Semi-supervised training and validation functions


In [ ]:
def accuracy(output, target, topk=(1,)):
    """
    Evaluates a model's top k accuracy

    Parameters:
        output (torch.autograd.Variable): model output
        target (torch.autograd.Variable): ground-truths/labels
        topk (list): list of integers specifying top-k precisions
            to be computed

    Returns:
        float: percentage of correct predictions
    """

    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
def train_supervised(train_loader, encoder, img_size, classifier, criterion, optimizer, device):
    """
    Trains/updates the classifier for one epoch on the training dataset.

    Parameters:
        train_loader (torch.utils.data.DataLoader): The trainset dataloader
        encoder (torch.nn.module): Fixed weight encoder
        img_size (int): Spatial size of the input images
        classifier (torch.nn.module): Classifier to be trained
        criterion (torch.nn.criterion): Loss function
        optimizer (torch.optim.optimizer): optimizer instance like SGD or Adam
        device (string): cuda or cpu
    """

    # create an instance of the average meter to track losses
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    # set the encoder to evaluate mode as it isn't trained 
    # (this would be important for e.g. dropout where stochasticity shouldn't be applied during testing)
    encoder.eval()
    classifier.train()

    # iterate through the dataset loader
    for i, (..., ...) in enumerate(train_loader):
        # transfer inputs and targets to the GPU (if it is available)
        inp = ...
        target = ...
        
        # compute output, i.e. the model forward
        # first flatten the image and calculate the static autoencoder's encoder
        inp = inp.view(...)
        embedding = encoder(...)
        # then compute the classifier on the obtained embedding
        output = classifier(...)
        
        # calculate the loss
        loss = criterion(..., ...)

        # measure accuracy and record loss and accuracy
        prec1, _ = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), inp.size(0))
        top1.update(prec1.item(), inp.size(0))

        # compute gradient and do the SGD step
        # we reset the optimizer with zero_grad to "flush" former gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print the loss every 100 mini-batches
        if i % 100 == 0:
            print('Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   loss=losses, top1=top1))

In [ ]:
def validate_supervised(val_loader, encoder, img_size, classifier, criterion, device):
    """
    Evaluates the model

    Parameters:
        val_loader (torch.utils.data.DataLoader): The valset dataloader
        encoder (torch.nn.module): Fixed weight encoder
        img_size (int): Spatial size of the input images
        classifier (torch.nn.module): Classifier to be trained
        criterion (torch.nn.criterion): Loss function
        device (string): cuda or cpu
    """

    # create instances of the average meter to track losses and accuracies
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode 
    # (this would be important for e.g. dropout where stochasticity shouldn't be applied during testing)
    encoder.eval()
    classifier.eval()

    # avoid computation of gradients and necessary storing of intermediate layer activations
    with torch.no_grad():
        # iterate through the dataset loader
        for i, (..., ...) in enumerate(val_loader):
            # transfer to device
            inp = ...
            target = ...

            # compute output, i.e. the model forward
            # first flatten the image and calculate the static autoencoder's encoder
            inp = inp.view(...)
            embedding = encoder(...)
            # then compute the classifier on the obtained embedding
            output = classifier(...)

            # compute loss
            loss = criterion(..., ...)

            # measure accuracy and record loss and accuracy
            prec1, _ = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), inp.size(0))
            top1.update(prec1.item(), inp.size(0))

    print(' * Validation accuracy: Prec@1 {top1.avg:.3f} '.format(top1=top1))

#### Semi-supervised optimization

In [ ]:
n_classes = len(dataset.val_loader.dataset.class_to_idx)

# Define the single linear layer classifier that maps from the encoder's latent 
# embedding to the amount of classes. 
classifier = ...

# Let us split the validation dataset into two separate sets, each containing 
# 50% of the data to use for supervised learning and validation. 
# We can use the torch.utils.data.random_split function for this purpose.
labeled_train_set, labeled_val_set = torch.utils.data.random_split(dataset.valset, [int(0.5 * len(dataset.valset)), int(0.5 * len(dataset.valset))])
supervised_train_loader = torch.utils.data.DataLoader(labeled_train_set, batch_size=batch_size, shuffle=True, 
                                                      num_workers=workers, pin_memory=is_gpu, sampler=None)
supervised_test_loader = torch.utils.data.DataLoader(labeled_val_set, batch_size=batch_size, shuffle=False, 
                                                     num_workers=workers, pin_memory=is_gpu, sampler=None)

# set the supervised loss function
criterion = ...

# create the optimizer. 
# IMPORTANT: we will only optimize the classifier parameters here and keep 
# the encoder parameters fixed to see how far we can get just by fine-tuning
# on a few labels. 
optimizer = torch.optim.Adam(...)

total_epochs = 40
for epoch in range(total_epochs):
    print("EPOCH:", epoch + 1)
    print("TRAIN")
    train_supervised(...)
    print("VALIDATION")
    validate_supervised(...)

### The pre-training clearly helped us to use only few labels to map the existing encoding to class concepts. However, our results seem to be much worse than what we likely expected. On the other hand, we have just used 5% of the labels of the dataset. 

## Moving from MLP to CNN
We will now implement the autoencoder for our previously used convolutional neural network. For downsampling operations such as pooling, we can analogously implement upsampling in the decoder. For strided convolution, we could add stride to the transposed version.

As previously mentioned we will implement the model through individual functions instead of nn.Sequential containers now. 

In [ ]:
class AE_CNN(nn.Module):
    def __init__(self):
        super(AE_CNN, self).__init__()
        
        self.encoder = nn.Sequential(nn.Conv2d(1, 64, 5), # input features, output features, kernel size 
                                     nn.ReLU(True), 
                                     nn.MaxPool2d(2, 2), # kernel size, stride 
                                     nn.Conv2d(64, 128, 5), # input features, output features, kernel size 
                                     nn.ReLU(True), 
                                     nn.MaxPool2d(2, 2)) # kernel size, stride
    
        # You can use the function nn.ConvTranspose2d() for the transposed convolutions
        # in the decoder. 
        
        # You can use the function nn.Upsample(scale_factor=) to upsample as the 
        # function to compensate the downsampling through pooling in the encoder.

        # Build a symmetric decoder 
        self.decoder = nn.Sequential(...)
    
    def forward(self, x):
        h = self.encoder(x)
        x = self.decoder(h)
        return x

In [ ]:
# create an instance of the MLP based autoencoder and transfer the model to the device.
# Note that we do not necessarily need any custom weight initialization as PyTorch
# already uses the initialization schemes that we have previously learned about internally. 
CNN_AE_model = AE_CNN().to(device)
# we can also print the model architecture
print(CNN_AE_model)

# set the loss function
criterion = nn.BCEWithLogitsLoss().to(device)

# we can use advanced stochastic gradient descent algorithms 
# with regularization (weight-decay) or momentum
optimizer = torch.optim.Adam(CNN_AE_model.parameters(), lr=1e-4)

total_epochs = 20
for epoch in range(total_epochs):
    print("EPOCH:", epoch + 1)
    print("TRAIN")
    train(...)
    print("VALIDATION")
    validate(...)

We can observe that already our first epoch achieves a better loss and qualitatively better looking reconstructed images in the convolutional neural network autoencoder variant.

As this encoding seems to be much better, we can ask ourselves the question whether our semi-supervised learning approach will fare much better now. This is something that you should try to get a better feeling for the learned representations. 



# Variational Autoencoder
We have seen how we can encode data into a latent vector and decode back to the original image for training with reconstruction loss.  

In the lecture we have encountered another Bayesian variant, called the variational autoencoder that provides a different perspective with a powerful generative model: https://arxiv.org/abs/1312.6114 . 
We will now minimize the variational lower bound on the evidence (ELBO). To approximate the true posterior to the generative model p(x, z) we will use our neural network encoder to calculate q(x|z) (the approximate posterior). This is sometimes also referred to as the recognition model.

The probabilistic decoder p(x|z) in turn computes the probability density of an input x under the generative model given a sample z from the approximate posterior q(z|x). Like in the autoencoder the parameters of encoder and decoder are optimized jointly. 

We can conduct this optimization using the reparameterization trick, where we express the random variable z through a deterministic variable as seen in the lecture.

In practical terms, in contrast to the regular autoencoder, we thus now optimize for both a reconstruction loss and KL divergence between our prior and approximate posterior. 

Following the lecture, we will use a Gaussian prior and implement the corresponding reparameterization.

A good in-depth introduction to variational auto-encoders for further reading can be found here: https://arxiv.org/abs/1906.02691

## Building our CNN model
Last week we have implemented our PyTorch models with nn.Sequential() containers in order to easily access the model.encoder to train a classifier on top. Prior to that we have seen how we can define our model's layers in the class' constructor and use a forward function to define the execution. This week we will encounter another third variant, where we define multiple custom functions called encode, decode and reparameterize which we can then individually access and define a joint forward function. 

In [ ]:
class VAE_CNN(nn.Module):
    def __init__(self, latent_dim):
        super(VAE_CNN, self).__init__()
        
        self.latent_dim = latent_dim 
        
        self.conv1 = nn.Conv2d(1, 64, 5) # input features, output features, kernel size
        self.mp1 = nn.MaxPool2d(2, 2) # kernel size, stride
        
        self.conv2 = nn.Conv2d(64, 128, 5) # input features, output features, kernel size
        self.mp2 = nn.MaxPool2d(2, 2) # kernel size, stride
        
        # tip: 4x4 is the remaining spatial resolution here
        self.latent_mu = ...
        self.latent_sigma = ...
        
        # decoder layers
        # implement the decoder layers 
        ...
        self.Upsample = nn.Upsample(scale_factor=2, mode='nearest')
        
    def encode(self, x):
        # define the probabilistic encoding
        ...
        return mu, sigma
        
    def reparameterize(self, mu, std):
        # define the reparameterization: z = eps * sigma + mu
        # where eps ~ N(0, 1)
        ...
        return z
    
    def decode(self, z):
        # define the probabilistic decoding
        ...
        return x
    
    def forward(self, x):
        # define the end-to-end forward function
        ...

        # make sure to return output, was well as latent mu and sigma to 
        # calculate the reconstruction and KLD loss terms
        return x, mu, sigma

#### We will now also have to define our loss function
In addition to the reconstruction loss, we need to implement the KL divergence between the approximate posterior, thus our latent mu and sigma and our unit Gaussian prior. 

We extend our loss function

In [ ]:
def VAE_loss_function(output, target, mu, std):
    recon_loss_func = nn.BCEWithLogitsLoss(reduction='sum')

    # compute reconstruction loss - normalize by batch size
    recon_loss = ...
    
    # numerical value for stability of log computation
    eps = 1e-8
    # compute Kullback Leibler Divergence - normalize by batch size
    kld = ...
    
    return recon_loss, kld

We will also need to modify our training and validation functions to include the modified loss and let the model return our latent mu and sigma vectors

In [ ]:
def train_VAE(train_loader, model, criterion, optimizer, device):
    """
    Trains/updates the model for one epoch on the training dataset.

    Parameters:
        train_loader (torch.utils.data.DataLoader): The trainset dataloader
        model (torch.nn.module): Model to be trained
        criterion (torch.nn.criterion): Loss function
        optimizer (torch.optim.optimizer): optimizer instance like SGD or Adam
        device (string): cuda or cpu
    """

    # create an instance of the average meter to track losses
    losses = AverageMeter()
    recon_losses = AverageMeter()
    kl_losses = AverageMeter()

    # switch to train mode
    model.train()

    # iterate through the dataset loader
    # we can now discard the labels returned by our old data loader as we no longer need them
    for i, (inp, _) in enumerate(train_loader):
        # transfer inputs and targets to the GPU (if it is available)
        inp = inp.to(device)
        target = inp
        
        # you can make your autoencoder a denoising autoencoder by 
        # adding noise to the input, but not to the target!
        # To test the denoising autoencoder, uncomment the line below, 
        # where we add a small Gaussian noise to the original images
        #inp = inp + torch.randn(inp.size()).to(device) * 0.1
        
        # compute output, i.e. the model forward. 
        # In contrast to our autoencoder this now also returns the latent mu
        # and sigma that we need to calculate the KL divergence
        ... = model(...)
        
        # calculate the loss
        recon_loss, kl_loss = criterion(...)
        loss = ...

        # record loss
        losses.update(loss.item(), inp.size(0))
        recon_losses.update(recon_loss.item(), inp.size(0))
        kl_losses.update(kl_loss.item(), inp.size(0))

        # compute gradient and do the SGD step
        # we reset the optimizer with zero_grad to "flush" former gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print the loss every 100 mini-batches
        if i % 100 == 0:
            print('Loss {loss.val:.4f} ({loss.avg:.4f}) \t'
                  'Reconstruction {recon.val:.4f} ({recon.avg:.4f}) \t'
                  'KLD {kld.val:.4f} ({kld.avg:.4f})'
                  .format(loss=losses, recon=recon_losses, kld=kl_losses))

In [ ]:
def validate_VAE(val_loader, model, criterion, device):
    """
    Trains/updates the model for one epoch on the training dataset.

    Parameters:
        val_loader (torch.utils.data.DataLoader): The valset dataloader
        model (torch.nn.module): Model to be trained
        criterion (torch.nn.criterion): Loss function
        device (string): cuda or cpu
    """

    # create an instance of the average meter to track losses
    losses = AverageMeter()
    recon_losses = AverageMeter()
    kl_losses = AverageMeter()

    # switch to train mode
    model.eval()

    # iterate through the dataset loader
    # we can now discard the labels returned by our old data loader as we no longer need them
    for i, (inp, _) in enumerate(val_loader):
        # transfer inputs and targets to the GPU (if it is available)
        inp = inp.to(device)
        target = inp
        
        # compute output, i.e. the model forward. 
        # In contrast to our autoencoder this now also returns the latent mu
        # and sigma that we need to calculate the KL divergence
        ... = model(...)
        
        # calculate the loss
        recon_loss, kl_loss = criterion(...)
        loss = ...

        # record loss
        losses.update(loss.item(), inp.size(0))
        recon_losses.update(recon_loss.item(), inp.size(0))
        kl_losses.update(kl_loss.item(), inp.size(0))
        
        # print the loss every 100 mini-batches
        if i % 100 == 0:
            print('Loss {loss.val:.4f} ({loss.avg:.4f}) \t'
                  'Reconstruction {recon.val:.4f} ({recon.avg:.4f}) \t'
                  'KLD {kld.val:.4f} ({kld.avg:.4f})'
                  .format(loss=losses, recon=recon_losses, kld=kl_losses))

In addition to training and validation, we can now also write a generation function. Once the model is training and minimizing the divergence between our approximate posterior distribution and our Gaussian prior, we can start sampling from the prior and directly generate images from the corresponding latent vector z. 

In [ ]:
def generate(model, device, num_images=100):
    with torch.no_grad():
        # sample from the Gaussian prior p(z)
        z = ...

        # decode the samples p(x|z)
        # add a sigmoid function at the end of the decoder to constrain 
        # the generated image to 0-1 range (which is otherwise not guaranteed)
        generated_images = ...

        # visualize
        imgs = torchvision.utils.make_grid(generated_images.cpu(),
                                           nrow=int(math.sqrt(generated_images.size(0))),
                                           padding=5)
        npimgs = imgs.numpy()
        # when using matplotlib the color channels are expected to be in the third 
        # instead of the first dimension -> tranpose
        plt.imshow(np.transpose(npimgs, (1,2,0)))
        plt.show()

### Constructing and running the CNN based VAE
Let's create an instance of our VAE model and optimize it. After each epoch we will sample from the prior and generate some example images and took a look at their visual quality.

In [ ]:
# create VAE model instance
latent_dim = 50 # like in the autoencoder, this is a hyper-parameter
# a good starting point for the above model is a latent dimension of 50.
model = VAE_CNN(latent_dim).to(device)
print(model)

# again, define loss function and optimizer. This time using our custom loss.
criterion = VAE_loss_function

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# optimize and visualize generations from generative model at each epoch
total_epochs = 20
for epoch in range(total_epochs):
    print("EPOCH:", epoch + 1)
    print("TRAIN")
    train_VAE(dataset.train_loader, model, criterion, optimizer, device)
    print("VALIDATION")
    validate_VAE(dataset.val_loader, model, criterion, device)
    print("GENERATION")
    generate(model, device, num_images=100)

We can see that initially the samples do not look like the data from our training distribution, but after some epochs of optimizing the generative model we are able to generate images by decoding the samples from our Gaussian prior. 

# Additional (optional) exercise

Use a 2 dimensional latent space and visualize the latent space of the trained model as well as traverse deterministic z values during generation to see how the model interpolates between different concepts. 